In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re

critical_keys1 = ['parity', 'paridy']
critical_keys2 = ['nc']

global_path = './outputs'

for c1 in critical_keys1:
    for c2 in critical_keys2:
        for c2_value in [True, False]:
            all_result_paths = glob.glob(global_path + '/pari*_exp/**/result.csv', recursive=True)
            all_result_paths = [p for p in all_result_paths if c1 in p]
            all_result_paths = [p for p in all_result_paths if (c2 in p)==c2_value]

            result_summary = {}
            n_layers = 6
            for result_path in all_result_paths:
                result = pd.read_csv(result_path)
                top_test_result = result['test_acc'].max()
                pe_type = 'NoPE' if 'nope' in result_path else 'Original'
                pattern = r"res=\[(.*?)\]"
                residual_list = re.findall(pattern, result_path)
                if len(residual_list) == 0:
                    residual_list = np.arange(n_layers)
                else:
                    residual_list = residual_list[0]
                    residual_list = [int(num) for num in residual_list.split(", ")] if residual_list!='' else []
                # residual_list = [int(num) for num in residual_list.split(", ")]
                missing_rc = [i for i in range(n_layers) if i not in residual_list]
                missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


                if pe_type not in result_summary:
                    result_summary[pe_type] = {}
                if missing_rc_str not in result_summary[pe_type]:
                    result_summary[pe_type][missing_rc_str] = []
                result_summary[pe_type][missing_rc_str].append(top_test_result)
                if pe_type == 'Original' and missing_rc_str == '{5}':
                    print(result_path, pe_type, missing_rc_str, top_test_result)

            from ast import literal_eval
            mean_results = {}
            for pe_type in result_summary:
                cur_type_dict = result_summary[pe_type].copy()
                for missing_rc in cur_type_dict:
                    cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
                mean_results[pe_type] = cur_type_dict

            df = pd.DataFrame(mean_results)
            missing0 = ['{}']
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            display(df.loc[missing0,:].T)
            display(df.loc[missing1,:].T)
            display(df.loc[missing2,:].T)
            display(df.loc[missing3,:].T)
            display(df.loc[missing4,:].T)
            display(df.loc[missing5,:].T)
            num_seeds = df.values.min()


            from collections import OrderedDict
            final_results = OrderedDict()
            for pe_type in {'Original', 'NoPE'}:
                cur_type_dict = result_summary[pe_type].copy()
                final_dict = {
                    'min': cur_type_dict.copy(),
                    'max': cur_type_dict.copy(),
                    'avg.': cur_type_dict.copy(),
                }
                for missing_rc in cur_type_dict:
                    final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
                    final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
                    final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

                for k in final_dict:
                    final_results[pe_type+' ('+k+')'] = final_dict[k]

            df = pd.DataFrame(final_results)
            missing1 = [str({i}) for i in range(n_layers)]
            missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
            missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
            missing4 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3)])+ '}'for i in range(n_layers-3)]
            missing5 = ['{' + ','.join([str(i), str(i+1), str(i+2), str(i+3), str(i+4)])+ '}'for i in range(n_layers-4)]
            from IPython.display import display
            r1 = df.loc[missing1,:].T
            r2 = df.loc[missing2,:].T
            r3 = df.loc[missing3,:].T
            r4 = df.loc[missing4,:].T
            r5 = df.loc[missing5,:].T
            r0 = df.loc[['{}'],:].T
            display(r0)
            display(r1)
            display(r2)
            display(r3)
            display(r4)
            display(r5)

            # write all 3 to the same csv file
            all_r = pd.concat([r0, r1, r2, r3, r4, r5], axis=1)
            c2 = '_' + c2 if c2_value else ''
            all_r.to_csv(f'{c1}{c2}_{num_seeds}seeds.csv')



./outputs/parity_nc_residual_exp/parity_nc_sd240_T2403281738_res=[0, 1, 2, 3, 4]_ncTrue/parity_nc_sd240_T2403281738_res=[0, 1, 2, 3, 4]_ncTrue/result.csv Original {5} 99.5


,{}
Original,1
NoPE,1


,{0},{1},{2},{3},{4},{5}
Original,1,1,1,1,1,1
NoPE,1,1,1,1,1,1


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original,1,1,1,1,1
NoPE,1,1,1,1,1


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original,1,1,1,1
NoPE,1,1,1,1


...

,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original (min),97.700,97.600,96.750,97.000,93.80
Original (max),98.050,98.800,98.800,99.300,94.90
Original (avg.),97.875,98.200,97.775,98.150,94.35
NoPE (min),54.100,52.050,51.300,50.300,50.15
NoPE (max),56.850,52.300,52.150,51.450,50.95
NoPE (avg.),55.475,52.175,51.725,50.875,50.55


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original (min),96.85,98.20,96.700,93.250
Original (max),98.80,98.80,99.050,94.100
Original (avg.),97.94,98.50,97.875,93.675
NoPE (min),54.65,50.85,50.200,49.900
NoPE (max),56.30,51.95,51.650,51.450
NoPE (avg.),55.87,51.40,50.925,50.675


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
Original (min),97.500,95.05,82.250
Original (max),98.450,98.15,83.700
Original (avg.),97.975,96.60,82.975
NoPE (min),54.350,50.45,50.450
NoPE (max),56.050,51.45,51.650
NoPE (avg.),55.200,50.95,51.050


,"{0,1,2,3,4}","{1,2,3,4,5}"
Original (min),96.700,83.500
Original (max),98.300,95.650
Original (avg.),97.500,89.575
NoPE (min),51.400,51.350
NoPE (max),51.450,52.050
NoPE (avg.),51.425,51.700


IndexError: list index out of range

,{}
NoPE (min),89.940
NoPE (max),96.870
NoPE (avg.),93.836
Original (min),98.180
Original (max),100.000
Original (avg.),99.326


,{0},{1},{2},{3},{4},{5}
NoPE (min),93.460,89.680,95.590,91.610,96.550,90.520
NoPE (max),98.770,100.000,100.000,100.000,100.000,99.190
NoPE (avg.),94.942,95.738,98.816,98.224,98.896,95.038
Original (min),88.770,87.210,99.260,99.970,99.930,81.490
Original (max),100.000,100.000,100.000,100.000,100.000,100.000
Original (avg.),95.138,94.486,99.778,99.994,99.986,95.520


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),90.920,92.180,97.400,93.680,91.310
NoPE (max),97.820,99.960,100.000,100.000,99.900
NoPE (avg.),95.016,96.802,99.048,98.100,97.278
Original (min),93.930,89.670,98.610,98.780,96.080
Original (max),99.370,100.000,100.000,100.000,99.500
Original (avg.),96.890,93.078,99.722,99.756,98.214


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),93.13,83.300,99.490,95.480
NoPE (max),99.50,95.690,100.000,100.000
NoPE (avg.),96.23,88.342,99.898,99.096
Original (min),93.51,83.460,96.980,100.000
Original (max),96.25,90.990,100.000,100.000
Original (avg.),95.29,87.640,99.396,100.000


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),82.550,86.160,94.300
NoPE (max),87.200,92.630,99.000
NoPE (avg.),84.896,89.006,97.192
Original (min),83.400,82.860,33.810
Original (max),88.930,90.440,97.480
Original (avg.),86.642,86.686,83.986


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),82.580,36.010
NoPE (max),90.800,90.720
NoPE (avg.),87.400,75.870
Original (min),85.710,80.840
Original (max),94.610,89.060
Original (avg.),90.006,84.248


In [5]:
# This is causal trained on mod3, 5 seeds

,{}
NoPE (min),100.000
NoPE (max),100.000
NoPE (avg.),100.000
Original (min),99.760
Original (max),100.000
Original (avg.),99.858


,{0},{1},{2},{3},{4},{5}
NoPE (min),98.930,79.780,98.440,99.980,98.620,100.0
NoPE (max),99.990,99.060,100.000,100.000,100.000,100.0
NoPE (avg.),99.590,88.108,99.684,99.996,99.720,100.0
Original (min),98.190,89.130,98.800,99.200,98.810,100.0
Original (max),100.000,100.000,100.000,100.000,100.000,100.0
Original (avg.),99.298,96.900,99.746,99.836,99.762,100.0


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),94.920,96.140,98.570,99.980,99.990
NoPE (max),100.000,99.930,100.000,100.000,100.000
NoPE (avg.),98.944,98.398,99.378,99.996,99.998
Original (min),99.520,98.250,99.930,99.670,99.710
Original (max),100.000,99.760,100.000,100.000,100.000
Original (avg.),99.824,99.058,99.986,99.928,99.906


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),99.940,79.720,77.960,33.350
NoPE (max),100.000,94.470,99.150,33.990
NoPE (avg.),99.974,89.012,92.662,33.670
Original (min),99.090,88.960,82.210,33.640
Original (max),100.000,94.820,98.740,33.990
Original (avg.),99.498,90.902,94.848,33.776


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),85.140,33.420,33.350
NoPE (max),94.920,34.490,33.990
NoPE (avg.),90.644,33.832,33.658
Original (min),63.880,33.480,33.510
Original (max),96.240,33.910,33.990
Original (avg.),78.078,33.708,33.750


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),33.580,33.350
NoPE (max),33.980,33.990
NoPE (avg.),33.736,33.658
Original (min),33.580,33.590
Original (max),33.910,33.990
Original (avg.),33.746,33.764


In [8]:
# This is non-causal trained on mod3, 2 seeds

,{}
NoPE (min),89.940
NoPE (max),95.460
NoPE (avg.),92.700
Original (min),98.540
Original (max),99.990
Original (avg.),99.265


,{0},{1},{2},{3},{4},{5}
NoPE (min),95.25,89.680,99.040,91.610,98.870,92.970
NoPE (max),98.77,99.400,99.480,100.000,100.000,95.260
NoPE (avg.),97.01,94.540,99.260,95.805,99.435,94.115
Original (min),88.77,87.210,99.740,100.000,100.000,99.020
Original (max),94.45,100.000,99.890,100.000,100.000,99.950
Original (avg.),91.61,93.605,99.815,100.000,100.000,99.485


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE (min),90.920,92.180,97.400,93.68,91.310
NoPE (max),93.870,95.380,100.000,99.18,99.450
NoPE (avg.),92.395,93.780,98.700,96.43,95.380
Original (min),93.930,91.190,98.610,100.00,98.130
Original (max),98.590,94.680,100.000,100.00,99.500
Original (avg.),96.260,92.935,99.305,100.00,98.815


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE (min),93.130000,83.570,99.490,95.48
NoPE (max),98.260000,93.940,100.000,100.00
NoPE (avg.),95.695000,88.755,99.745,97.74
Original (min),93.510000,83.460,96.980,100.00
Original (max),96.250000,90.990,100.000,100.00
Original (avg.),95.326667,87.225,98.490,100.00


,"{0,1,2,3}","{1,2,3,4}","{2,3,4,5}"
NoPE (min),83.090,86.76,98.270
NoPE (max),87.200,92.12,99.000
NoPE (avg.),85.145,89.44,98.635
Original (min),86.570,82.86,33.810
Original (max),87.630,90.44,97.480
Original (avg.),86.960,86.60,75.460


,"{0,1,2,3,4}","{1,2,3,4,5}"
NoPE (min),82.58,36.010000
NoPE (max),86.98,90.720000
NoPE (avg.),84.78,63.365000
Original (min),85.71,81.720000
Original (max),91.76,89.060000
Original (avg.),88.68,85.413333
